In [1]:
import pandas as pd

In [2]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_21.csv')

In [3]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [4]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [5]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [6]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [7]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [8]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,ATL,6/21/2021,1,660670,Ronald Acuna,1,Y,RF
1,ATL,6/21/2021,1,518692,Freddie Freeman,2,Y,1B
2,ATL,6/21/2021,1,645277,Ozzie Albies,3,Y,2B
3,ATL,6/21/2021,1,501659,Abraham Almonte,4,Y,LF
4,ATL,6/21/2021,1,663586,Austin Riley,5,Y,3B
...,...,...,...,...,...,...,...,...
145,SD,6/21/2021,1,543333,Eric Hosmer,5,N,NaN
146,SD,6/21/2021,1,571976,Wil Myers,6,N,NaN
147,SD,6/21/2021,1,595777,Jurickson Profar,7,N,NaN
148,SD,6/21/2021,1,593590,Webster Rivas,8,N,NaN


In [9]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [10]:
pitch_df.drop(456, inplace = True)

In [11]:
matchups

{'NYM': 'ATL',
 'ATL': 'NYM',
 'BAL': 'HOU',
 'HOU': 'BAL',
 'CHC': 'CLE',
 'CLE': 'CHC',
 'TEX': 'OAK',
 'OAK': 'TEX',
 'MIN': 'CIN',
 'CIN': 'MIN',
 'ARI': 'MIL',
 'MIL': 'ARI',
 'SD': 'LAD',
 'LAD': 'SD'}

In [12]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    if len(lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values) > 0: 
        opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                                & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
        if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
                opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
                opp_pitcher.append(opp_pitch)
                keep_pitch.append(opp_pitch)
        else:
            del_indexes.append(i)
    else:
        del_indexes.append(i)
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-12-a4b56b56add5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-12-a4b56b56add5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [13]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-13-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [14]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,ATL,6/21/2021,1,660670,Ronald Acuna,1,Y,RF,R,Jacob deGrom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ATL,6/21/2021,1,518692,Freddie Freeman,2,Y,1B,R,Jacob deGrom,...,0.320690,0.448179,0.634483,1.082662,0.313793,0.324324,82.199191,38.788347,0.445850,181.648864
2,ATL,6/21/2021,1,645277,Ozzie Albies,3,Y,2B,R,Jacob deGrom,...,0.245536,0.310484,0.468750,0.779234,0.223214,0.266272,33.801912,4.037927,0.332809,108.137575
3,ATL,6/21/2021,1,501659,Abraham Almonte,4,Y,LF,R,Jacob deGrom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ATL,6/21/2021,1,663586,Austin Riley,5,Y,3B,R,Jacob deGrom,...,0.273408,0.359477,0.456929,0.816406,0.183521,0.320652,46.533674,9.426494,0.353500,121.225782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,SD,6/21/2021,1,665487,Fernando Tatis,4,N,NaN,L,Julio Urias,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,SD,6/21/2021,1,543333,Eric Hosmer,5,N,NaN,L,Julio Urias,...,0.280899,0.326531,0.483146,0.809677,0.202247,0.291667,14.110250,2.210542,0.344980,119.612085
146,SD,6/21/2021,1,571976,Wil Myers,6,N,NaN,L,Julio Urias,...,0.263158,0.345794,0.557895,0.903689,0.294737,0.281250,18.653075,5.552131,0.377963,140.561062
147,SD,6/21/2021,1,595777,Jurickson Profar,7,N,NaN,L,Julio Urias,...,0.255102,0.305556,0.367347,0.672902,0.112245,0.278481,11.308315,-2.052621,0.293091,84.754696


In [15]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [16]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [17]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [18]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,ATL,6/21/2021,1,660670,Ronald Acuna,1,Y,RF,R,Jacob deGrom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ATL,6/21/2021,1,518692,Freddie Freeman,2,Y,1B,R,Jacob deGrom,...,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
2,ATL,6/21/2021,1,645277,Ozzie Albies,3,Y,2B,R,Jacob deGrom,...,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
3,ATL,6/21/2021,1,501659,Abraham Almonte,4,Y,LF,R,Jacob deGrom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ATL,6/21/2021,1,663586,Austin Riley,5,Y,3B,R,Jacob deGrom,...,15.142857,15.142857,15.142857,15.142857,15.142857,15.142857,15.142857,15.142857,15.142857,15.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,SD,6/21/2021,1,665487,Fernando Tatis,4,N,NaN,L,Julio Urias,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,SD,6/21/2021,1,543333,Eric Hosmer,5,N,NaN,L,Julio Urias,...,9.818182,9.818182,9.818182,9.818182,9.818182,9.818182,9.818182,9.818182,9.818182,9.818182
146,SD,6/21/2021,1,571976,Wil Myers,6,N,NaN,L,Julio Urias,...,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500
147,SD,6/21/2021,1,595777,Jurickson Profar,7,N,NaN,L,Julio Urias,...,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500,8.437500


In [19]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [20]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [21]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
17,NYM,6/21/2021,1,594798,Jacob deGrom,SP,Y,NaN,2,6,...,2.571429,2.765217,0.156522,2.608696,1.000000,4.095238,1.773913,4.639175,4.236202,7.055722
27,HOU,6/21/2021,1,543606,Jake Odorizzi,SP,Y,NaN,3,6,...,18.692308,1.421053,0.236842,1.184211,1.690141,7.961538,1.750000,2.692308,37.654455,25.914861
37,BAL,6/21/2021,1,669211,Keegan Akin,SP,Y,NaN,4,5,...,6.887755,1.310345,0.379310,0.931034,1.297710,6.887755,1.611111,4.102564,19.671004,21.853788
46,CLE,6/21/2021,1,650644,Aaron Civale,SP,N,NaN,4,4,...,5.734513,0.792208,0.168831,0.623377,1.065744,4.778761,1.192661,2.953368,17.161743,15.945209
55,CHC,6/21/2021,1,640470,Adbert Alzolay,SP,N,NaN,3,5,...,4.354839,1.531250,0.343750,1.187500,0.862069,4.354839,1.117021,4.320988,14.917106,14.882839
65,OAK,6/21/2021,1,593423,Frankie Montas,SP,N,NaN,3,6,...,9.529412,1.303371,0.359551,0.943820,1.500000,7.508021,1.700000,4.365325,26.779470,25.436622
75,TEX,6/21/2021,1,502043,Kyle Gibson,SP,N,NaN,4,5,...,4.256757,1.186709,0.474684,0.712025,1.219081,6.689189,1.542289,3.850325,18.531263,18.185145
103,CIN,6/21/2021,1,641816,Tyler Mahle,SP,Y,NaN,2,7,...,9.219512,2.048035,0.397380,1.650655,1.716346,8.195122,2.181159,5.143369,24.435692,23.950022
113,MIN,6/21/2021,1,457918,JA Happ,SP,N,NaN,2,7,...,11.259574,1.328313,0.548193,0.780121,1.744186,9.025532,1.881057,5.203837,34.642088,36.722908
122,MIL,6/21/2021,1,474463,Brett Anderson,SP,N,NaN,3,5,...,6.553398,0.716667,0.300000,0.416667,1.423358,6.990291,1.504425,3.941176,23.629441,22.776965


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [22]:
scores = pd.read_csv('../datasets/6_21_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210621,110a,506433,"Darvish, Yu",1.0,9,1.0,35.70,10200.0,SDG,v lad,NaN,6,2
1,20210621,114t,543606,"Odorizzi, Jake",1.0,0,1.0,32.65,5900.0,HOU,@ bal,NaN,10,2
2,20210621,121n,594798,"deGrom, Jacob",1.0,9,NaN,25.45,NaN,NYM,v atl,NaN,4,2(F7)
3,20210621,151s,518876,"Kelly, Merrill",1.0,9,1.0,24.15,7400.0,ARI,v mil,NaN,5,1
4,20210621,162d,666120,"Anderson, Ian",1.0,9,NaN,23.60,NaN,ATL,@ nym,2.0,1,0(F7)


In [23]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [24]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [25]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [26]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [27]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [28]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [29]:
bat_final.to_csv('../merged_data/hitters_6_21.csv')
pitch_final.to_csv('../merged_data/pitchers_6_21.csv')

In [30]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
2,ATL,6/21/2021,1,518692,Freddie Freeman,2,Y,1B,R,Jacob deGrom,...,1.0,2,NaN,4.0,NaN,ATL,@ nym,NaN,2,4(F7)
4,ATL,6/21/2021,1,645277,Ozzie Albies,3,Y,2B,R,Jacob deGrom,...,1.0,3,NaN,16.0,NaN,ATL,@ nym,NaN,2,4(F7)
9,ATL,6/21/2021,1,663586,Austin Riley,5,Y,3B,R,Jacob deGrom,...,1.0,5,NaN,0.0,NaN,ATL,@ nym,NaN,2,4(F7)
11,ATL,6/21/2021,1,621020,Dansby Swanson,6,Y,SS,R,Jacob deGrom,...,1.0,6,NaN,0.0,NaN,ATL,@ nym,NaN,2,4(F7)
12,ATL,6/21/2021,1,628338,Guillermo Heredia,7,Y,CF,R,Jacob deGrom,...,1.0,7,NaN,2.0,NaN,ATL,@ nym,NaN,2,4(F7)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,LAD,6/21/2021,1,621035,Chris Taylor,6,N,NaN,R,Yu Darvish,...,1.0,6,4.0,2.0,5200.0,LAD,@ sdg,NaN,2,6
101,LAD,6/21/2021,1,656716,Zach McKinstry,8,N,NaN,R,Yu Darvish,...,1.0,7,57.0,0.0,3300.0,LAD,@ sdg,NaN,2,6
102,SD,6/21/2021,1,592518,Manny Machado,2,N,NaN,L,Julio Urias,...,1.0,4,5.0,29.0,4500.0,SDG,v lad,NaN,6,2
103,SD,6/21/2021,1,630105,Jake Cronenworth,3,N,NaN,L,Julio Urias,...,1.0,3,4.0,27.0,5100.0,SDG,v lad,NaN,6,2


In [31]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,HOU,6/21/2021,1,543606,Jake Odorizzi,SP,Y,NaN,3,6,...,1.0,0,1.0,32.65,5900.0,HOU,@ bal,NaN,10,2
1,BAL,6/21/2021,1,669211,Keegan Akin,SP,Y,NaN,4,5,...,1.0,0,1.0,3.00,5100.0,BAL,v hou,NaN,2,10
2,CLE,6/21/2021,1,650644,Aaron Civale,SP,N,NaN,4,4,...,1.0,9,1.0,17.50,8500.0,CLE,@ chc,NaN,4,0
3,CHC,6/21/2021,1,640470,Adbert Alzolay,SP,N,NaN,3,5,...,1.0,9,1.0,12.10,8900.0,CHC,v cle,NaN,0,4
4,OAK,6/21/2021,1,593423,Frankie Montas,SP,N,NaN,3,6,...,1.0,0,1.0,1.55,9400.0,OAK,@ tex,NaN,3,8
5,TEX,6/21/2021,1,502043,Kyle Gibson,SP,N,NaN,4,5,...,1.0,0,1.0,13.20,6300.0,TEX,v oak,NaN,8,3
6,CIN,6/21/2021,1,641816,Tyler Mahle,SP,Y,NaN,2,7,...,1.0,0,1.0,13.60,9800.0,CIN,@ min,NaN,5,7(F12)
7,MIN,6/21/2021,1,457918,JA Happ,SP,N,NaN,2,7,...,1.0,0,1.0,6.50,5300.0,MIN,v cin,NaN,7,5(F12)
8,MIL,6/21/2021,1,474463,Brett Anderson,SP,N,NaN,3,5,...,1.0,9,1.0,-4.00,6500.0,MIL,@ ari,NaN,1,5
9,ARI,6/21/2021,1,518876,Merrill Kelly,SP,N,NaN,4,4,...,1.0,9,1.0,24.15,7400.0,ARI,v mil,NaN,5,1
